# Converting saved_model to TFLite

In [1]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization, Embedding
from tensorflow.keras import layers

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# load the end to end model
e2e_model_path = '/content/drive/MyDrive/saved_model/end-to-end'
e2e_model = tf.keras.models.load_model(e2e_model_path)

In [6]:
# converter_e2e = tf.lite.TFLiteConverter.from_saved_model(export_dir_e2e)
converter_e2e = tf.lite.TFLiteConverter.from_keras_model(e2e_model)
converter_e2e.optimizations = [tf.lite.Optimize.DEFAULT]
converter_e2e.experimental_new_converter=True
converter_e2e.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS]

tflite_e2emodel = converter_e2e.convert()
open('/content/drive/MyDrive/Github/toxicity-classifier-model/end_to_end_model.tflite', 'wb').write(tflite_e2emodel)

4737592

## Testing the TFLite model

In [7]:
# load tflite model
interpreter = tf.lite.Interpreter(model_content=tflite_e2emodel)
interpreter.allocate_tensors()

# get input and output tensor details (name, shape, data type, index, quantization params)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print('input shape:', input_details[0]['shape'])
print('input type:', input_details[0]['dtype'])
print('output shape:', output_details[0]['shape'])
print('output type:', output_details[0]['dtype'])

input shape: [1 1]
input type: <class 'numpy.bytes_'>
output shape: [1 6]
output type: <class 'numpy.float32'>


In [10]:
input_text = input("Enter the input text: ")
print()

# preprocess input text
input_text_bytes = bytes(input_text, 'utf-8')
input_tensor = np.array([input_text_bytes], dtype=object)
input_tensor = np.expand_dims(input_tensor, axis=1) # add dimension to match input shape

# set input tensor
interpreter.set_tensor(input_details[0]['index'], input_tensor)

# run inference
interpreter.invoke()

# get output tensor
output_tensor = interpreter.get_tensor(output_details[0]['index'])
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
print("End-to-end tflite prediction ")
for label, prediction in zip(labels, output_tensor.flatten()):
    print(label + ": ", prediction)

Enter the input text: i will smack your face

End-to-end tflite prediction 
toxic:  0.9909723
severe_toxic:  0.075705096
obscene:  0.30569848
threat:  0.76839155
insult:  0.17350985
identity_hate:  0.0033001879
